In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [3]:
items           = pd.read_csv('items.csv',usecols=["item_id", "item_category_id"])
item_categories = pd.read_csv('item_categories.csv')
shops           = pd.read_csv('shops.csv')
sales_train     = pd.read_csv('sales_train.csv.gz')
test            = pd.read_csv('test.csv.gz')

In [4]:
sales_train[['day','month', 'year']] = sales_train['date'].str.split('.', expand=True).astype(int)
sales_train = sales_train[sales_train['year'].isin([2013]) == False]
sales_train = sales_train.set_index('item_id').join(items.set_index('item_id'))
sales_train.reset_index(inplace=True)

In [5]:
# Якутск Орджоникидзе, 56
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [294]:
max_sales=1000
sums = sales_train.groupby('item_id')['item_cnt_day'].sum().reset_index().rename(columns={"item_cnt_day":"item_total_sales"}).sort_values(by='item_total_sales')

ids_reject = sums[(sums['item_total_sales'] > 0) & (sums['item_total_sales'] < max_sales)]['item_id'].unique()

In [6]:
train_item_ids = sales_train['item_id'].unique()
#train_item_ids = np.setdiff1d(train_item_ids, ids_reject)
train_shop_ids = sales_train['shop_id'].unique()
test_item_ids = test['item_id'].unique()
test_shop_ids = test['shop_id'].unique()
train_blocks = sales_train['date_block_num'].unique()

all_item_ids = np.unique(np.append(test_item_ids,train_item_ids))
all_shop_ids = np.unique(np.append(train_shop_ids,test_shop_ids))


In [7]:
combinations = []

for dbn in range(np.min(train_blocks), np.max(train_blocks)+1):
    sales = sales_train[sales_train.date_block_num==dbn]
    #item_ids = np.intersect1d(sales.item_id.unique(), test_item_ids)
    item_ids = all_item_ids
    #dbn_combos = list(product(sales.shop_id.unique(), item_ids, [dbn]))
    dbn_combos = list(product(all_shop_ids, item_ids, [dbn]))
    for combo in dbn_combos:
        combinations.append(combo)
        
all_combos = pd.DataFrame(np.unique(np.vstack([combinations]), axis=0), columns=['shop_id','item_id','date_block_num'])

In [8]:
ys = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_item_cnt_block"})

training = all_combos.merge(ys, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)


training['shop_item_cnt_block'] = training['shop_item_cnt_block'].clip(0,20).astype('int8')

training = training.set_index('item_id').join(items.set_index('item_id'))
training.reset_index(inplace=True)

for col in ['item_id', 'shop_id', 'item_category_id']:
    training[col] = pd.to_numeric(training[col], downcast='unsigned')

In [9]:
dates = sales_train[['date_block_num', 'month', 'year']].drop_duplicates(['date_block_num', 'month', 'year'])

dates_dict = {}

for index,row in dates.iterrows():
    dates_dict[row['date_block_num']] = {"month": row['month'], "year": row['year']}
    
training['month'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['month']), downcast='unsigned')
training['year'] = pd.to_numeric(training['date_block_num'].apply(lambda block: dates_dict[block]['year']), downcast='unsigned')



In [10]:
training = training[(~training['date_block_num'].isin([22,23]))]

In [ ]:
training.date_block_num.unique()

array([12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30,
       31, 32, 33])

In [11]:
ys = sales_train.groupby(['item_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"item_cnt_block"})

training = training.merge(ys, on=['item_id', 'date_block_num'], how='left').fillna(0)

ys = sales_train.groupby(['shop_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_cnt_block"})

training = training.merge(ys, on=['shop_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"category_cnt_block"})


training = training.merge(ys, on=['item_category_id', 'date_block_num'], how='left').fillna(0)


ys = sales_train.groupby(['shop_id', 'item_category_id', 'date_block_num'], as_index=False)['item_cnt_day']\
                .sum().rename(columns={"item_cnt_day":"shop_category_cnt_block"})

training = training.merge(ys, on=['shop_id', 'item_category_id', 'date_block_num'], how='left').fillna(0)


In [121]:
a = training.groupby(['shop_id','date_block_num'])['shop_cnt_block'].sum()\
                        .reset_index()
a = a[a['shop_cnt_block'] == 0]
closed_shop_ids = a["shop_id"].unique()
closed_shop_ids = np.setdiff1d(closed_shop_ids, [36])
print(closed_shop_ids)


#with pd.option_context('display.max_rows', 999):
    #print(a[a['item_cnt_block'] == 0])

training = training[~training['shop_id'].isin(closed_shop_ids)]

[ 9 13 17 20 27 29 30 33 34 39 40 43 48 51 54]


In [12]:
training['item_cnt_block_mean'] = training.groupby(['item_id','year'])['item_cnt_block'].transform(np.mean)
training['item_cnt_block_min'] = training.groupby(['item_id','year'])['item_cnt_block'].transform(np.min)
training['item_cnt_block_max'] = training.groupby(['item_id','year'])['item_cnt_block'].transform(np.max)
training['item_cnt_block_std'] = training.groupby(['item_id','year'])['item_cnt_block'].transform(np.std)
training['item_cnt_block_med'] = training.groupby(['item_id','year'])['item_cnt_block'].transform(np.median)

In [13]:
training['shop_cnt_block_mean'] = training.groupby(['shop_id','year'])['shop_cnt_block'].transform(np.mean)
training['shop_cnt_block_min'] = training.groupby(['shop_id','year'])['shop_cnt_block'].transform(np.min)
training['shop_cnt_block_max'] = training.groupby(['shop_id','year'])['shop_cnt_block'].transform(np.max)
training['shop_cnt_block_std'] = training.groupby(['shop_id','year'])['shop_cnt_block'].transform(np.std)
training['shop_cnt_block_med'] = training.groupby(['shop_id','year'])['shop_cnt_block'].transform(np.median)

In [14]:
training['category_cnt_block_mean'] = training.groupby(['item_category_id','year'])['category_cnt_block'].transform(np.mean)
training['category_cnt_block_min'] = training.groupby(['item_category_id','year'])['category_cnt_block'].transform(np.min)
training['category_cnt_block_max'] = training.groupby(['item_category_id','year'])['category_cnt_block'].transform(np.max)
training['category_cnt_block_std'] = training.groupby(['item_category_id','year'])['category_cnt_block'].transform(np.std)
training['category_cnt_block_med'] = training.groupby(['item_category_id','year'])['category_cnt_block'].transform(np.median)

In [15]:
training['shop_category_cnt_block_mean'] = training.groupby(['shop_id', 'item_category_id','year'])['shop_category_cnt_block'].transform(np.mean)
training['shop_category_cnt_block_min'] = training.groupby(['shop_id', 'item_category_id','year'])['shop_category_cnt_block'].transform(np.min)
training['shop_category_cnt_block_max'] = training.groupby(['shop_id', 'item_category_id','year'])['shop_category_cnt_block'].transform(np.max)
training['shop_category_cnt_block_std'] = training.groupby(['shop_id', 'item_category_id','year'])['shop_category_cnt_block'].transform(np.std)
training['shop_category_cnt_block_med'] = training.groupby(['shop_id', 'item_category_id','year'])['shop_category_cnt_block'].transform(np.median)

In [16]:
training['shop_item_cnt_block_mean'] = training.groupby(['shop_id', 'item_id','year'])['shop_item_cnt_block'].transform(np.mean)
training['shop_item_cnt_block_min'] = training.groupby(['shop_id', 'item_id','year'])['shop_item_cnt_block'].transform(np.min)
training['shop_item_cnt_block_max'] = training.groupby(['shop_id', 'item_id','year'])['shop_item_cnt_block'].transform(np.max)
training['shop_item_cnt_block_std'] = training.groupby(['shop_id', 'item_id','year'])['shop_item_cnt_block'].transform(np.std)
training['shop_item_cnt_block_med'] = training.groupby(['shop_id', 'item_id','year'])['shop_item_cnt_block'].transform(np.median)

In [283]:

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
training.sample(10)

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,month,year,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,shop_item_cnt_block_mean,shop_item_cnt_block_min,shop_item_cnt_block_max,shop_item_cnt_block_std,shop_item_cnt_block_med,item_cnt_block_mean,item_cnt_block_min,item_cnt_block_max,item_cnt_block_std,item_cnt_block_med,shop_cnt_block_mean,shop_cnt_block_min,shop_cnt_block_max,shop_cnt_block_std,shop_cnt_block_med,category_cnt_block_mean,category_cnt_block_min,category_cnt_block_max,category_cnt_block_std,category_cnt_block_med,shop_category_cnt_block_mean,shop_category_cnt_block_min,shop_category_cnt_block_max,shop_category_cnt_block_std,shop_category_cnt_block_med
5573325,22087,31,17,0,83,6,2014,33.0,7701.0,105.0,0.0,0.0,0,0,0.000000,0.0,40.2,14.0,83.0,20.966021,32.5,8615.6,7056.0,10017.0,915.964756,8520.0,150.2,46.0,309.0,92.624683,105.5,0.0,0.0,0.0,0.000000,0.0
4033932,15488,9,26,0,63,3,2015,0.0,0.0,2076.0,0.0,0.3,0,3,0.948683,0.0,5.8,0.0,14.0,5.311520,6.0,318.5,-1.0,3186.0,955.842604,0.0,1880.9,1316.0,2555.0,408.558703,1932.0,38.5,0.0,385.0,115.543446,0.0
843518,3443,3,32,4,30,9,2015,143.0,745.0,5303.0,171.0,1.6,0,4,1.264911,2.0,79.0,20.0,224.0,64.096865,59.0,715.8,535.0,977.0,108.940300,734.5,7487.0,4150.0,14751.0,2860.250222,6519.5,106.5,55.0,210.0,46.286811,89.5
3674254,14252,6,28,0,55,5,2015,5.0,1748.0,6353.0,192.0,0.3,0,2,0.674949,0.0,7.6,4.0,13.0,3.385443,6.0,1850.9,1484.0,2329.0,288.380469,1775.0,6895.7,4913.0,9291.0,1348.040733,6413.5,228.7,156.0,310.0,47.964073,226.5
2760521,10930,4,13,0,55,2,2014,11.0,1299.0,9442.0,124.0,0.2,0,1,0.421637,0.0,14.5,8.0,20.0,4.228843,14.5,1241.5,1090.0,1430.0,111.587142,1229.5,10110.4,8364.0,11533.0,1016.016085,9965.0,160.8,123.0,217.0,29.299962,168.0
3963479,15295,53,33,0,63,10,2015,1.0,1061.0,2149.0,27.0,0.2,0,1,0.421637,0.0,2.7,1.0,5.0,1.346610,3.0,1293.8,1061.0,1747.0,181.719007,1287.5,1880.9,1316.0,2555.0,408.558703,1932.0,24.4,15.0,34.0,6.202332,27.0
5361023,21021,55,15,0,61,4,2014,0.0,1271.0,94.0,0.0,0.0,0,0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1676.2,1067.0,4229.0,915.542975,1313.0,354.3,94.0,1784.0,484.460616,188.5,0.0,0.0,0.0,0.000000,0.0
2746195,10889,47,29,0,55,6,2015,7.0,1740.0,6017.0,138.0,0.2,0,1,0.421637,0.0,9.4,3.0,17.0,4.413170,8.5,1947.8,1700.0,2410.0,207.579362,1902.5,6895.7,4913.0,9291.0,1348.040733,6413.5,157.4,116.0,205.0,26.768389,153.5
2356913,9354,21,27,0,70,4,2015,0.0,2053.0,1966.0,66.0,0.0,0,0,0.000000,0.0,4.7,0.0,47.0,14.113074,0.0,1823.8,1622.0,2053.0,124.217784,1841.5,1835.6,1024.0,2369.0,372.475927,1917.0,43.4,30.0,66.0,11.391891,40.5
1496339,5822,31,33,20,35,10,2015,259.0,6112.0,685.0,66.0,20.0,20,20,0.000000,20.0,412.8,211.0,843.0,211.592699,316.5,6765.9,5714.0,8815.0,893.551734,6416.0,994.6,568.0,2279.0,516.819406,724.5,87.1,58.0,159.0,34.186367,68.5


In [18]:
training.describe()

,item_id,shop_id,date_block_num,shop_item_cnt_block,item_category_id,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,month,item_cnt_block_mean,item_cnt_block_min,item_cnt_block_max,item_cnt_block_std,item_cnt_block_med,shop_cnt_block_mean,shop_cnt_block_min,shop_cnt_block_max,shop_cnt_block_std,shop_cnt_block_med,category_cnt_block_mean,category_cnt_block_min,category_cnt_block_max,category_cnt_block_std,category_cnt_block_med,shop_category_cnt_block_mean,shop_category_cnt_block_min,shop_category_cnt_block_max,shop_category_cnt_block_std,shop_category_cnt_block_med
count,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06,2.148720e+06
mean,1.101830e+04,3.166667e+01,2.850000e+01,2.388669e-01,4.627815e+01,1.257451e+01,1.659483e+03,3.773882e+03,8.193124e+01,5.500000e+00,1.257451e+01,3.116302e+00,4.066224e+01,1.149766e+01,8.791634e+00,1.659483e+03,1.307952e+03,2.294381e+03,2.985570e+02,1.594798e+03,3.773882e+03,2.520990e+03,5.851549e+03,9.875356e+02,3.444695e+03,8.193124e+01,5.224146e+01,1.262866e+02,2.226545e+01,7.812230e+01
std,6.246577e+03,1.753319e+01,2.872282e+00,1.068823e+00,1.676899e+01,8.034849e+01,1.400722e+03,3.751384e+03,1.708449e+02,2.872282e+00,6.182224e+01,2.927392e+01,1.920728e+02,5.007974e+01,5.266674e+01,1.324073e+03,1.081161e+03,1.904318e+03,3.460083e+02,1.270089e+03,3.522610e+03,2.405506e+03,5.108302e+03,8.299933e+02,3.245611e+03,1.602227e+02,1.067848e+02,2.707611e+02,5.498987e+01,1.506967e+02
min,3.000000e+01,2.000000e+00,2.400000e+01,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,-1.000000e+00,-1.000000e+00,1.000000e+00,-1.000000e-01,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.300000e+01,0.000000e+00,3.300000e+02,3.450961e+01,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e-01,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.394250e+03,1.600000e+01,2.600000e+01,0.000000e+00,3.700000e+01,0.000000e+00,9.130000e+02,6.800000e+02,3.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,4.000000e+00,1.184627e+00,0.000000e+00,9.083000e+02,7.320000e+02,1.233000e+03,1.203324e+02,8.960000e+02,7.871000e+02,4.460000e+02,1.622000e+03,3.046295e+02,6.540000e+02,4.000000e+00,0.000000e+00,1.100000e+01,3.049556e+00,2.500000e+00
50%,1.119050e+04,3.450000e+01,2.850000e+01,0.000000e+00,4.300000e+01,2.000000e+00,1.251500e+03,2.203000e+03,3.100000e+01,5.500000e+00,4.200000e+00,0.000000e+00,1.100000e+01,3.138223e+00,2.000000e+00,1.298450e+03,9.885000e+02,1.706500e+03,1.830122e+02,1.303000e+03,1.880900e+03,1.316000e+03,4.824000e+03,9.188205e+02,1.917000e+03,3.180000e+01,1.600000e+01,5.700000e+01,1.162392e+01,3.000000e+01
75%,1.606725e+04,4.700000e+01,3.100000e+01,0.000000e+00,5.800000e+01,1.000000e+01,1.719000e+03,6.353000e+03,1.000000e+02,8.000000e+00,1.160000e+01,2.000000e+00,3.000000e+01,8.593942e+00,8.500000e+00,1.807200e+03,1.320000e+03,2.329000e+03,2.777342e+02,1.699500e+03,6.895700e+03,4.913000e+03,9.291000e+03,1.348041e+03,6.413500e+03,1.031000e+02,6.400000e+01,1.460000e+02,2.391182e+01,1.000000e+02
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,8.300000e+01,7.869000e+03,8.815000e+03,1.510900e+04,5.129000e+03,1.000000e+01,3.860600e+03,1.924000e+03,7.869000e+03,2.118527e+03,3.512000e+03,6.765900e+03,5.714000e+03,8.815000e+03,1.809498e+03,6.416000e+03,1.017430e+04,6.779000e+03,1.510900e+04,2.860258e+03,9.293500e+03,1.575700e+03,1.079000e+03,5.129000e+03,1.531763e+03,1.502500e+03


In [284]:
training.columns.values

array(['item_id', 'shop_id', 'date_block_num', 'shop_item_cnt_block',
       'item_category_id', 'month', 'year', 'item_cnt_block',
       'shop_cnt_block', 'category_cnt_block', 'shop_category_cnt_block',
       'shop_item_cnt_block_mean', 'shop_item_cnt_block_min',
       'shop_item_cnt_block_max', 'shop_item_cnt_block_std',
       'shop_item_cnt_block_med', 'item_cnt_block_mean',
       'item_cnt_block_min', 'item_cnt_block_max', 'item_cnt_block_std',
       'item_cnt_block_med', 'shop_cnt_block_mean', 'shop_cnt_block_min',
       'shop_cnt_block_max', 'shop_cnt_block_std', 'shop_cnt_block_med',
       'category_cnt_block_mean', 'category_cnt_block_min',
       'category_cnt_block_max', 'category_cnt_block_std',
       'category_cnt_block_med', 'shop_category_cnt_block_mean',
       'shop_category_cnt_block_min', 'shop_category_cnt_block_max',
       'shop_category_cnt_block_std', 'shop_category_cnt_block_med'],
      dtype=object)

In [17]:
features = [
    
    'item_cnt_block', 'shop_cnt_block',
       'category_cnt_block', 'shop_category_cnt_block',
       'item_cnt_block_mean', 'item_cnt_block_min', 'item_cnt_block_max',
       'item_cnt_block_std', 'item_cnt_block_med', 'shop_cnt_block_mean',
       'shop_cnt_block_min', 'shop_cnt_block_max', 'shop_cnt_block_std',
       'shop_cnt_block_med', 'category_cnt_block_mean',
       'category_cnt_block_min', 'category_cnt_block_max',
       'category_cnt_block_std', 'category_cnt_block_med',
       'shop_category_cnt_block_mean', 'shop_category_cnt_block_min',
       'shop_category_cnt_block_max', 'shop_category_cnt_block_std',
       'shop_category_cnt_block_med','shop_item_cnt_block_std'

]

In [18]:
training.fillna(0,inplace=True)

In [19]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler 


training[features] = StandardScaler().fit_transform(training[features])



In [20]:
training[features].sample(10)

,item_cnt_block,shop_cnt_block,category_cnt_block,shop_category_cnt_block,item_cnt_block_mean,item_cnt_block_min,item_cnt_block_max,item_cnt_block_std,item_cnt_block_med,shop_cnt_block_mean,...,category_cnt_block_min,category_cnt_block_max,category_cnt_block_std,category_cnt_block_med,shop_category_cnt_block_mean,shop_category_cnt_block_min,shop_category_cnt_block_max,shop_category_cnt_block_std,shop_category_cnt_block_med,shop_item_cnt_block_std
18075786,-0.098065,-0.204718,-0.869314,-0.466321,-0.103646,-0.037082,-0.113147,-0.124672,-0.083633,-0.214823,...,-0.902259,-0.784372,-0.601515,-0.917605,-0.489304,-0.429904,-0.503731,-0.479708,-0.485127,-0.272058
16193663,-0.041047,-0.644996,2.313891,-0.049741,-0.010225,-0.037082,-0.046500,-0.025318,-0.000254,-0.648145,...,2.160073,2.058827,1.886281,2.128340,0.032216,0.047407,-0.038185,-0.106879,0.044404,0.382193
9119903,0.091995,0.048315,2.313891,0.840880,0.118785,0.022331,0.153442,0.139471,0.106947,0.125177,...,2.160073,2.058827,1.886281,2.128340,0.663767,0.347430,0.722095,0.819467,0.689279,0.600277
14667398,0.091995,3.290299,-0.402144,-0.466321,0.007570,-0.037082,0.029668,0.087256,-0.071721,1.011629,...,-0.201654,-0.199077,-0.170224,-0.198364,-0.489304,-0.429904,-0.503731,-0.479708,-0.485127,-0.272058
11189921,-0.098065,-0.231919,-0.931674,-0.466321,-0.114767,-0.037082,-0.160752,-0.180436,-0.083633,-0.285775,...,-0.925007,-1.034471,-1.082551,-0.954050,-0.489304,-0.429904,-0.503731,-0.479708,-0.485127,-0.272058
12020947,-0.098065,-0.974571,-0.809434,-0.466321,-0.112543,-0.037082,-0.151231,-0.169283,-0.083633,-1.034031,...,-0.791421,-0.913907,-0.984517,-0.805755,-0.489304,-0.429904,-0.503731,-0.479708,-0.485127,-0.272058
4819226,-0.098065,-0.321746,-0.869491,-0.442380,-0.110319,-0.037082,-0.151231,-0.165566,-0.083633,-0.298712,...,-0.842242,-0.846756,-0.795241,-0.864049,-0.444588,-0.395810,-0.443444,-0.399886,-0.448427,-0.272058
8848362,-0.098065,0.388644,2.835803,2.535932,-0.105870,-0.037082,-0.132189,-0.146364,-0.083633,0.425557,...,2.160073,2.058827,1.886281,2.128340,2.599116,2.481689,2.440260,2.117294,2.631769,0.382193
2574239,-0.022041,-0.156642,-0.196462,-0.001858,2.038360,0.052038,5.361444,6.062989,0.321349,-0.082219,...,0.079314,1.033476,1.757482,0.306458,0.390446,0.231512,0.608220,0.578586,0.332763,8.722416
8088144,-0.079059,-0.562760,0.339801,-0.279578,-0.103646,-0.037082,-0.122668,-0.135670,-0.083633,-0.538787,...,0.278000,0.045555,-0.190361,0.226123,-0.305415,-0.266254,-0.326220,-0.321090,-0.285898,-0.272058


In [21]:
a = training[['shop_item_cnt_block'] + ['date_block_num','item_id', 'shop_id', 'year'] + features].sort_values(by=["date_block_num"]).groupby(["item_id", "shop_id", "year"])#['item_id_mean_encoding']#[features+['item_cnt_block']]

In [152]:
len(training['date_block_num'].unique())

22

In [156]:
sorted(training['date_block_num'].unique())[1:22][-1]

33

In [22]:
#groups
lstm_data = []
lstm_y = []

for name, group in a:
    #print(group.values)
    steps = []
    ys = []
    #print("group.values",group.values)
    for step in group.values:
        #print(step)
        #print("step", len(step))
        #step is np.array 
        #step[4:] is np.array print(type(step[4:]))
        steps.append(step[5:])
        #print(step[9])
        #print(step[0])
        ys.append(step[0])
    #remove last
    #print(type(steps[0:8]))
    #jan to sept, y = oct
    lstm_data.append(np.array(steps[0:8]))
    #remove first
    #print(ys)
    #preditct for setptember
    lstm_y.append(ys[-1])
    
lstm_y = np.array([np.array([y]) for y in lstm_y])

In [23]:
small_data = np.array(lstm_data)
small_y = np.array(lstm_y)

In [24]:
small_data.shape

(1882656, 8, 25)

In [32]:
print(len(lstm_y))

print(len([y for y in lstm_y if y == 0]))

zeros_indices = []
for idx,y in enumerate(lstm_y):
    if y == 0:
        zeros_indices.append(idx)

1882656
1808680


In [33]:
lstm_data_no_zeros = []
for idx,sample in enumerate(lstm_data):
    if idx not in zeros_indices:
        lstm_data_no_zeros.append(sample)

KeyboardInterrupt: 

In [ ]:
lstm_y_no_zeros = []
for idx,y in enumerate(lstm_y):
    if idx not in zeros_indices:
        lstm_y_no_zeros.append(y)

In [25]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(small_data, small_y, test_size=0.1, random_state=42)

In [26]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


#x_train_scaled = MinMaxScaler().fit_transform(x_train[features])


#x_reshaped = np.reshape(x_train_scaled, (x_train_scaled.shape[0], 10, x_train_scaled.shape[1]))
    
#x_val_scaled_reshaped = np.reshape(x_val_scaled, (x_val_scaled.shape[0], 1, x_val_scaled.shape[1]))


callbacks = [
    EarlyStopping(monitor='val_loss', patience=1, verbose=0)
]


my_model = Sequential()
#bi directional?
my_model.add(LSTM(use_bias = True,unit_forget_bias=True,units = 16, dropout=0.5,recurrent_dropout=0.5,input_shape = (small_data.shape[1],len(features))))
my_model.add(Dropout(0.5))
#my_model.add(BatchNormalization())
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()


#lstmd_dataa = np.array(np.array(lstm_data)).reshape(271148,8,51)
#print(lstmd_dataa.shape)
#print(lstmd_dataa)
#lstm_yy = np.array([np.array([y]) for y in lstm_y[0:100]])
#print(lstm_yy.shape)
#print(lstm_yy)

history = my_model.fit(train_data, y_train, batch_size=64, epochs=100,
                      validation_data=(val_data,y_val), callbacks=callbacks
                      )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

import math
print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 16)                2688      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 2,705
Trainable params: 2,705
Non-trainable params: 0
_________________________________________________________________
Train on 1694390 samples, validate on 188266 samples
Epoch 1/100
1694390/1694390 [==============================] - 496s 293us/step - loss: 0.3608 - mean_squared_error: 0.3608 - val_loss: 0.2274 - val_mean_squared_error: 0.2274
Epoch 2/100
  93440/1694390 [>.............................] - ETA: 7:47 - loss: 0.2991 - mean_squared_error: 0.2991

KeyboardInterrupt: 

In [169]:
training_test = training[(training['shop_id'].isin(test['shop_id'].unique())) & (training['item_id'].isin(test['item_id'].unique()))]

In [174]:
len(test_lstm_data[0])

21

In [175]:
test_lstm_data = []

for index, row in test.iterrows():
    if index % 10000 == 0:
        print(index)
    line = training_test[(training_test['shop_id'] == row['shop_id']) & (training_test['item_id'] == row['item_id'])].sort_values(by=['date_block_num'])[features].values
    test_lstm_data.append(line[1:22])

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000


In [ ]:
training_test[(training_test['shop_id'] == 5) & (training_test['item_id'] == 5037)].sort_values(by=['date_block_num'])

In [176]:
preds = my_model.predict(np.array(test_lstm_data),batch_size=len(test_lstm_data))
preds.clip(0,20,out=preds)

array([[0.20451248],
       [0.01601815],
       [0.31358206],
       ...,
       [0.05747372],
       [0.        ],
       [0.26262653]], dtype=float32)

In [177]:

print(np.mean(preds))
print(np.max(preds))

0.20561361
10.890563


In [178]:
submission = test.loc[:,['ID']]
submission['item_cnt_month'] = preds

submission.to_csv('submission.csv', index=False)

In [66]:
bestpreds = pd.read_csv('submissionbest.csv')['item_cnt_month']
print(np.mean(bestpreds))
print(np.max(bestpreds))

0.3542884257097005
16.49019305497366


In [ ]:
lr_preds = pd.read_csv('lr110.csv')['item_cnt_month']
lg_preds = pd.read_csv('lg110.csv')['item_cnt_month']
#cb_preds = pd.read_csv('cb102.csv')['item_cnt_month']


#preds = np.mean(np.array([lr_preds, lg_preds]),axis=0)

preds = (lg_preds * 0.50) + (lr_preds * 0.50)